In [3]:
# Получение данных
import pandas as pd

file_name = './Plus/data/casePlus729.csv'  # Откуда брать файл
data_source = pd.read_csv(file_name, sep=';', index_col=0,
                          skiprows=[1]  # Пропуск единиц измерения
                          ).dropna(
    axis=1, how='all')  # Чтение файла

column_names = data_source.columns.values
# pd.options.display.float_format = '{:,.2f}'.format # Если нужно ограничить вывод до 2 знаков после запятой

# Разбиение таблицы на параметры и критерии
params = data_source.iloc[:, 0:6].astype(float)
criteria1_column = data_source.iloc[:, 6].astype(float)
criteria2_column = data_source.iloc[:, 7].astype(float)
criteria3_column = data_source.iloc[:, 8].astype(float)


In [4]:
# Линейная интерполяция
from scipy.interpolate import LinearNDInterpolator
import numpy as np

# Создание точек для интерполяции
points = np.array(params.values)

PRINT_PROGRESS = False  # Вывод прогресса
# Верхние и нижние границы параметров
# X_LOWER = np.array([240, 340, 90, 1.3, 60, 110])
# X_UPPER = np.array([260, 360, 110, 2, 80, 130])
X_LOWER = params.min().to_numpy()
X_UPPER = params.max().to_numpy()

print('Нижняя граница:', X_LOWER)
print('Верхняя граница:', X_UPPER)

# Создание интерполяторов
print('Расчет интерполяторов...')
print(f'Расчет интерполятора {column_names[6]}...')
criteria1 = LinearNDInterpolator(points, criteria1_column.values)
print(f'Расчет интерполятора {column_names[7]}...')
criteria2 = LinearNDInterpolator(points, criteria2_column.values)
print(f'Расчет интерполятора {column_names[8]}...')
criteria3 = LinearNDInterpolator(points, criteria3_column.values)
print('Расчет интерполяторов завершен')



Нижняя граница: [2.4e+02 4.0e+02 9.0e+01 1.3e+00 2.5e-01 2.5e-01]
Верхняя граница: [2.6e+02 4.2e+02 1.1e+02 2.0e+00 3.0e-01 3.0e-01]
Расчет интерполяторов...
Расчет интерполятора DIM KMOL/HR...
Расчет интерполятора METHANOL KMOL/HR...
Расчет интерполятора Q KJ/HR...
Расчет интерполяторов завершен


In [5]:
# Однокритериальная оптимизация
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch  # Алгоритм Хука-Дживса
from pymoo.core.problem import Problem
from pymoo.optimize import minimize


class Best_c1 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = -criteria1(x)


class Best_c2 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria2(x)


class Best_c3 (Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = criteria3(x)


res_c1 = minimize(Best_c1(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
res_c2 = minimize(Best_c2(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)
res_c3 = minimize(Best_c3(), algorithm=PatternSearch(),
                  seed=1, verbose=PRINT_PROGRESS)

print('Однокритериальная оптимизация для каждого критерия (Метод Хука-Дживс)')
print(f'Первый критерий: {column_names[6]}')
print('X: ', res_c1.X)
print('F: ', res_c1.F)

print(f'Второй критерий: {column_names[7]}')
print('X: ', res_c2.X)
print('F: ', res_c2.F)

print(f'Третий критерий: {column_names[8]}')
print('X: ', res_c3.X)
print('F: ', res_c3.F)

# Сохранение результатов для однокритериальной оптимизации на основе маргинальных критериев
best_c1 = -res_c1.F
best_c2 = res_c2.F
best_c3 = res_c3.F



Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False

Однокритериальная оптимизация для каждого критерия (Метод Хука-Дживс)
Первый критерий: DIM KMOL/HR
X:  [2.60000000e+02 4.18030462e+02 9.59854313e+01 1.30000000e+00
 3.00000000e-01 3.00000000e-01]
F:  [-99.5506035]
Второй критерий: METHANOL KMOL/HR
X:  [2.43140881e+02 4.10424689e+02 1.02074276e+02 1.30000000e+00
 3.00000000e-01 2.50000000e-01]
F:  [24.1105337]
Третий критерий: Q KJ/HR
X:  [2.40276639e+02 4.00000000e+02 1.06206213e+02 1.30000000e+00
 2.50000000e-01 2.50000000e-01]
F:  [24091629.7]


In [27]:
# Оптимизация на основе маргинальных решений

from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from pymoo.algorithms.soo.nonconvex.pattern import PatternSearch  # Алгоритм Хука-Дживса


def objective_function(x):
    '''Маргинальная целевая функция'''
    сurrent_c1 = criteria1(x)
    сurrent_c2 = criteria2(x)
    сurrent_c3 = criteria3(x)

    diff_c1 = best_c1 - сurrent_c1
    diff_c2 = сurrent_c2 - best_c2
    diff_c3 = сurrent_c3 - best_c3

    return np.sqrt(diff_c1**2 + diff_c2**2 + diff_c3**2)


# Постановка задачи однокритериальной оптимизации
class Single_objective_optimization(Problem):
    def __init__(self) -> None:
        super().__init__(n_var=6, n_obj=1, n_constr=0, xl=X_LOWER, xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = objective_function(x)


single_objective_problem = Single_objective_optimization()

# Запуск оптимизации
try:
    # Хука - Дживс
    res_PS = minimize(
        single_objective_problem, algorithm=PatternSearch(), seed=1)
    print('PS')
    print(f'X: {res_PS.X}')
    print('F: ', res_PS.F)
    print('C1: ', criteria1(res_PS.X))
    print('C2: ', criteria2(res_PS.X))
    print('C3: ', criteria3(res_PS.X))

except Exception as e:
    print('Ошибка PS')
    print(e)


PS
X:  [2.40276639e+02 4.00000000e+02 1.04956213e+02 1.30000000e+00
 2.50000000e-01 2.50000000e-01]
F:  [18.892634]
C1:  [80.6579695]
C2:  [24.110877]
C3:  [24091629.7]


In [28]:
# Многокритериальная оптимизация
from pymoo.optimize import minimize
from pymoo.core.problem import Problem

# Алгоритмы оптимизации
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.moo.age import AGEMOEA

# Постановка задачи


class Multi_objective_optimization(Problem):
    def __init__(self):
        super().__init__(n_var=6, n_obj=3, n_constr=0,
                         # Минимальные значения параметров
                         xl=X_LOWER,
                         # Максимальные значения параметров
                         xu=X_UPPER)

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = -1 * criteria1(x)
        f2 = criteria2(x)
        f3 = criteria3(x)
        out["F"] = np.column_stack([f1, f2, f3])


# Запуск оптимизации
problem = Multi_objective_optimization()

try:
    print('Многокритериальная оптимизация (Метод NSGA2)')
    res_NSGA2 = minimize(problem, algorithm=NSGA2(), seed=1,
                         verbose=PRINT_PROGRESS, termination=('n_gen', 50))  # Вывод прогресса)
    # print('X: ', res_NSGA2.X)
    # print('F: ', res_NSGA2.F)

    # Сохранение результатов
    res_NSGA2_df = pd.DataFrame(res_NSGA2.X)
    res_NSGA2_df['dimEther_MCMF'] = -1 * res_NSGA2.F[:, 0]
    res_NSGA2_df['methanol_MCMF'] = res_NSGA2.F[:, 1]
    res_NSGA2_df['q_summ'] = res_NSGA2.F[:, 2]
    res_NSGA2_df.to_csv('MOO NSGA2 Results.csv', sep=',', index=False,
                        header=data_source.columns.values)

except Exception as e:
    print('Ошибка NSGA2')
    print(e)

try:
    print('Многокритериальная оптимизация (Метод AGEMOEA)')
    res_AGEMOEA = minimize(problem, algorithm=AGEMOEA(),
                           seed=1, verbose=PRINT_PROGRESS)

    # print('X: ', res_AGEMOEA.X)
    # print('F: ', res_AGEMOEA.F)

    # Сохранение результатов
    res_AGEMOEA_df = pd.DataFrame(res_AGEMOEA.X)
    res_AGEMOEA_df['dimEther_MCMF'] = -1 * res_AGEMOEA.F[:, 0]
    res_AGEMOEA_df['methanol_MCMF'] = res_AGEMOEA.F[:, 1]
    res_AGEMOEA_df['q_summ'] = res_AGEMOEA.F[:, 2]
    res_AGEMOEA_df.to_csv('MOO AGEMOEA Results.csv', sep=',', index=False,
                          header=data_source.columns.values)
except Exception as e:
    print('Ошибка AGEMOEA')
    print(e)


Многокритериальная оптимизация (Метод NSGA2)
Многокритериальная оптимизация (Метод AGEMOEA)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pymoo/algorithms/moo/age.py:290: RuntimeWarning: divide by zero encountered in true_divide
  normalization = 1. / hyperplane


KeyboardInterrupt: 